# Download and Cache features of Kinetics 700 (top 20 classes)
### Movivation
I need split this datase because is so big. For validate model with any data is so dificult, the duration of train is big and very expensive. I resolve split the data into 20 classes, select top classes with more samples, and into this samples, I split for get 40 videos for train, and 20 for validate.

In [3]:
!python -m pip install git+https://github.com/pytube/pytube
!pip install numpy pandas tensorflow moviepy Pillow videoio matplotlib scikit-learn

  Cloning https://github.com/pytube/pytube to /tmp/pip-req-build-z8ya7f4m
  Created wheel for pytube: filename=pytube-10.5.0-py3-none-any.whl size=43307 sha256=b97ecec0086be29b70a428679a9af626e67f928dc5b7d8325792e99acbbc3576
  Stored in directory: /tmp/pip-ephem-wheel-cache-kfen44_n/wheels/a8/ac/8c/337af6a10cc543c5eadf4eb2bbd02bd8609b25bea729df338a
Successfully built pytube


In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf

from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from PIL import Image
import videoio
import pytube

import matplotlib.pyplot as plt
from sklearn import datasets

import shutil
import time
import os

## Constants

In [5]:
# name of file for save from download 
OUTPUT_DOWNLOAD = 'kinetics700_2020.tag.gz'
# name of folder that extract
OUTPUT_EXTRACT = 'kinetics700_2020'

# number of samples by label for train
TRAIN_SAMPLES_BY_LABEL = 120
# number of samoles by label for validate
VALIDATE_SAMPLES_BY_LABEL = 20

ABSOLUTE_BASE_PATH = 'kinetics'
VIDEOS_BASE_PATH = f'{ABSOLUTE_BASE_PATH}/videos'
FEATURES_BASE_PATH = f'{ABSOLUTE_BASE_PATH}/features'

# paths for train and validate videos
VIDEOS_PATH_TRAIN = f'{VIDEOS_BASE_PATH}/train'
VIDEOS_PATH_VALIDATE = f'{VIDEOS_BASE_PATH}/validate'

# paths for train and validate features
FEATURES_PATH_TRAIN = f'{FEATURES_BASE_PATH}/train'
FEATURES_PATH_VALIDATION = f'{FEATURES_BASE_PATH}/validation'

# path for train csv
CSV_TRAIN_PATH = f'{OUTPUT_EXTRACT}/train.csv'
# path for validate csv
CSV_VALIDATE_PATH = f'{OUTPUT_EXTRACT}/validate.csv'

## Download and unzip dataset
Download from google storage api: `https://storage.googleapis.com/deepmind-media/Datasets/kinetics700_2020.tar.gz`, dataset are called: `kinetics700_2020`.

In [6]:
# download using curl
_ = os.system(f'curl https://storage.googleapis.com/deepmind-media/Datasets/kinetics700_2020.tar.gz \
        --output {OUTPUT_DOWNLOAD}')

# using the operation system (unix like) for unzip
_ = os.system(f'tar -xf {OUTPUT_DOWNLOAD}')

## Create datafrom from csv
Create `data_train` for train and `data_validate` for validate

In [7]:
data_train = pd.read_csv(CSV_TRAIN_PATH, sep=',')
data_validate = pd.read_csv(CSV_VALIDATE_PATH, sep=',')

In [8]:
data_train.head()

,label,youtube_id,time_start,time_end,split
0,clay pottery making,---0dWlqevI,19,29,train
1,news anchoring,---aQ-tA5_A,9,19,train
2,using bagging machine,---j12rm3WI,14,24,train
3,javelin throw,--07WQ2iBlw,1,11,train
4,climbing a rope,--0NTAs-fA0,29,39,train


In [9]:
data_validate.head()

,label,youtube_id,time_start,time_end,split
0,testifying,---QUuC4vJs,84,94,validate
1,washing feet,--GkrdYZ9Tc,0,10,validate
2,air drumming,--nQbRBEz2s,104,114,validate
3,pull ups,--rd8woSLiM,41,51,validate
4,building cabinet,--uGS0Y4D6k,9,19,validate


## Extract top 20 labels from train dataset

In [10]:
top_20_labels = data_train['label'].value_counts()[:20]

In [11]:
# show all labels and how many items it have
top_20_labels

hurling (sport)        994
shearing sheep         994
snowkiting             994
crawling baby          993
reading book           993
sailing                993
finger snapping        993
brushing teeth         991
riding camel           990
blowing out candles    990
skiing slalom          989
playing bagpipes       989
opening present        989
biking through snow    988
parasailing            988
playing drums          988
pole vault             988
shaking head           987
kitesurfing            987
capoeira               987
Name: label, dtype: int64

## How many samples have into 20 classes
This improve that split this data (kinetics700_2020) into 20 classes are not suficient

In [12]:
top_20_labels.sum()

19805

Maybe 20k of samples are expencive for validate e NN model...

## Get top 20 classes from datasets
For train and validate

In [13]:
data_train_labed = data_train[data_train['label'].isin(top_20_labels.index.to_list())]
data_validate_labed = data_validate[data_validate['label'].isin(top_20_labels.index.to_list())]

In [14]:
data_train_labed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19805 entries, 22 to 543448
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   label       19805 non-null  object
 1   youtube_id  19805 non-null  object
 2   time_start  19805 non-null  int64 
 3   time_end    19805 non-null  int64 
 4   split       19805 non-null  object
dtypes: int64(2), object(3)
memory usage: 928.4+ KB


In [15]:
data_validate_labed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 982 entries, 81 to 34080
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   label       982 non-null    object
 1   youtube_id  982 non-null    object
 2   time_start  982 non-null    int64 
 3   time_end    982 non-null    int64 
 4   split       982 non-null    object
dtypes: int64(2), object(3)
memory usage: 46.0+ KB


## Split samples by class
Make a group by label, before, get head with split number from agrouped dataframe.

In [16]:
data_train_labed = data_train_labed.groupby('label')
data_validate_labed = data_validate_labed.groupby('label')

In [17]:
data_train_labed_head = data_train_labed.head(TRAIN_SAMPLES_BY_LABEL)
data_validate_labed_head = data_validate_labed.head(VALIDATE_SAMPLES_BY_LABEL)

In [18]:
data_train_labed_head.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2400 entries, 22 to 88827
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   label       2400 non-null   object
 1   youtube_id  2400 non-null   object
 2   time_start  2400 non-null   int64 
 3   time_end    2400 non-null   int64 
 4   split       2400 non-null   object
dtypes: int64(2), object(3)
memory usage: 112.5+ KB


In [19]:
data_validate_labed_head.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 81 to 19263
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   label       400 non-null    object
 1   youtube_id  400 non-null    object
 2   time_start  400 non-null    int64 
 3   time_end    400 non-null    int64 
 4   split       400 non-null    object
dtypes: int64(2), object(3)
memory usage: 18.8+ KB


with this, i have: 800 samples for train (dividing into 20 classes) and 400 for validate (...). Maybe 1200 samples for using during the train.

## Create donwloader class
This class help to download and trim video

In [47]:
class Downloader:
    def __init__(self):
        self.proxies = {'socks5':'socks5://83.241.6.224:31567'}
    
    def trim(self, row, label_to_dir, test=False):
        label = row['label'] if not test else ''
        filename = row['youtube_id']
        time_start = row['time_start']
        time_end = row['time_end']

        input_filename = os.path.join(label_to_dir['tmp'], f'{filename}.mp4')
        output_filename = os.path.join(label_to_dir[label], f'{filename}.mp4')

        if os.path.exists(output_filename):
            print('Already trimmed: ', filename)
        else:
            print('Start trimming: ', filename)

            try:
                ffmpeg_extract_subclip(input_filename, time_start, time_end, targetname=output_filename)
                os.remove(input_filename)
            except Exception as e:
                print(f'Error in trimming: {e}')

            finally:
                print('Finish trimming: ', filename)

        return output_filename

    def download_clip(self, row, label_to_dir, test=False):
        filename = row['youtube_id']

        label = row['label'] if not test else ''
        output_filename = os.path.join(label_to_dir[label], f'{filename}.mp4')
        if os.path.exists(output_filename):
            print('Already download and trimming: ', filename)
            return False

        if not os.path.exists(os.path.join(label_to_dir['tmp'], filename + '.mp4')):
            print('Start downloading: ', filename)
            try:
                pytube.YouTube('https://www.youtube.com/watch?v=' + filename, proxies=self.proxies) \
                    .streams \
                    .get_lowest_resolution() \
                    .download(label_to_dir['tmp'], filename)
                print('Finish downloading: ', filename)
                return True
            except Exception as e:
                print(f'Error in download video: {e}')
                return False
        else:
            print('Already downloaded: ', filename)
            return True

    def execute(self, row, label, test=False):
        if self.download_clip(row, label, test):
            return self.trim(row, label, test)

In [48]:
downloader = Downloader()

## Create folders for save videos
The folders contains the `tmp` folder, using it for download full video, trim video from there and save on target path, by class.

In [49]:
folders_names = top_20_labels.index.to_list() + ['tmp']

In [50]:
def create_file_structure(path, folders_names):
    mapping = {}
    if not os.path.exists(path):
        os.mkdir(path)
    for name in folders_names:
        dir_ = os.path.join(path, name)
        if not os.path.exists(dir_):
            os.mkdir(dir_)
        mapping[name] = dir_
    return mapping

### Create base paths

In [51]:
def create_path(path):
    if not os.path.exists(path):
        os.mkdir(path)

In [52]:
create_path(ABSOLUTE_BASE_PATH)
create_path(VIDEOS_BASE_PATH)
create_path(FEATURES_BASE_PATH)

create_path(VIDEOS_PATH_TRAIN)
create_path(VIDEOS_PATH_VALIDATE)

create_path(FEATURES_PATH_TRAIN)
create_path(FEATURES_PATH_VALIDATION)

In [53]:
train_label_to_dir = create_file_structure(VIDEOS_PATH_TRAIN, folders_names)
validate_label_to_dir = create_file_structure(VIDEOS_PATH_VALIDATE, folders_names)

## Download train videos
after finish videos download, I delete tmp path.

In [54]:
for _, row in data_train_labed_head.iterrows():
    downloader.execute(row, train_label_to_dir)
shutil.rmtree(train_label_to_dir['tmp'])

Already download and trimming:  --8FQVwWH0M
Already download and trimming:  --IPbe5ZMCI
Already download and trimming:  --Kr1PZaPDI
Already download and trimming:  --RVKBMp_2M
Already download and trimming:  --SrV9bGzRA
Already download and trimming:  --XR9XONO2U
Already download and trimming:  --d6UCSGoHg
Already downloaded:  --feYHvtteA
Start trimming:  --feYHvtteA
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command returned an error
Error in trimming: ffmpeg version 4.2.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enabl

KeyboardInterrupt: 

## Download train videos
after finish videos download, I delete tmp path.

In [50]:
for _, row in data_validate_labed_head.iterrows():
    downloader.execute(row, validate_label_to_dir)
shutil.rmtree(validate_label_to_dir['tmp'])

Start downloading:  -BpPSxp00_M
Finish downloading:  -BpPSxp00_M
Start trimming:  -BpPSxp00_M
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  -BpPSxp00_M
Start downloading:  -RdGKu6-iFY
Finish downloading:  -RdGKu6-iFY
Start trimming:  -RdGKu6-iFY
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  -RdGKu6-iFY
Start downloading:  -UdK0if_jB4
Finish downloading:  -UdK0if_jB4
Start trimming:  -UdK0if_jB4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  -UdK0if_jB4
Start downloading:  02g7pYEXJm0
Finish downloading:  02g7pYEXJm0
Start trimming:  02g7pYEXJm0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  02g7pYEXJm0
Start downloading:  05Sy2LShZtM
Finish downloading:  05Sy2LShZtM
Start trimming:  05Sy2LShZtM
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  05Sy2LShZtM
Start downloading:  09dQut5GJ68
Fin

Finish downloading:  7cdCwXC8-og
Start trimming:  7cdCwXC8-og
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  7cdCwXC8-og
Start downloading:  7dqyrkaUvok
Finish downloading:  7dqyrkaUvok
Start trimming:  7dqyrkaUvok
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  7dqyrkaUvok
Start downloading:  7fwaq0mIkm4
Finish downloading:  7fwaq0mIkm4
Start trimming:  7fwaq0mIkm4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  7fwaq0mIkm4
Start downloading:  7j9Q839JiYY
Finish downloading:  7j9Q839JiYY
Start trimming:  7j9Q839JiYY
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  7j9Q839JiYY
Start downloading:  7l8Z6vijamc
Finish downloading:  7l8Z6vijamc
Start trimming:  7l8Z6vijamc
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  7l8Z6vijamc
Start downloading:  7oaqAyMa0b8
Finish downloading:  7oaqAyMa0b8
St

Finish downloading:  8nB66zNz29s
Start trimming:  8nB66zNz29s
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  8nB66zNz29s
Start downloading:  8qhiHVjYr-A
Finish downloading:  8qhiHVjYr-A
Start trimming:  8qhiHVjYr-A
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  8qhiHVjYr-A
Start downloading:  8ualLg1Ti1A
Finish downloading:  8ualLg1Ti1A
Start trimming:  8ualLg1Ti1A
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  8ualLg1Ti1A
Start downloading:  90rL37Eboz0
Finish downloading:  90rL37Eboz0
Start trimming:  90rL37Eboz0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  90rL37Eboz0
Start downloading:  91I8FgSgbw4
Finish downloading:  91I8FgSgbw4
Start trimming:  91I8FgSgbw4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  91I8FgSgbw4
Start downloading:  92MJ0AtBVJ4
Finish downloading:  92MJ0AtBVJ4
St

Finish downloading:  C33uuLgJY5Y
Start trimming:  C33uuLgJY5Y
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  C33uuLgJY5Y
Start downloading:  C3oqFeMPpXA
Finish downloading:  C3oqFeMPpXA
Start trimming:  C3oqFeMPpXA
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  C3oqFeMPpXA
Start downloading:  C4CeYm3SoUI
Finish downloading:  C4CeYm3SoUI
Start trimming:  C4CeYm3SoUI
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  C4CeYm3SoUI
Start downloading:  C4LPMCzihM0
Finish downloading:  C4LPMCzihM0
Start trimming:  C4LPMCzihM0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  C4LPMCzihM0
Start downloading:  C5RC-O8yPN4
Finish downloading:  C5RC-O8yPN4
Start trimming:  C5RC-O8yPN4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  C5RC-O8yPN4
Start downloading:  C62TGaw59e4
Finish downloading:  C62TGaw59e4
St

Error in download video: Dt9UZuXnggM is unavailable
Start downloading:  E0hkhMzYK04
Finish downloading:  E0hkhMzYK04
Start trimming:  E0hkhMzYK04
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  E0hkhMzYK04
Start downloading:  E2S_7S2yQdE
Finish downloading:  E2S_7S2yQdE
Start trimming:  E2S_7S2yQdE
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  E2S_7S2yQdE
Start downloading:  E2nfGGYuMXE
Finish downloading:  E2nfGGYuMXE
Start trimming:  E2nfGGYuMXE
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  E2nfGGYuMXE
Start downloading:  E4NJxD5tlqI
Finish downloading:  E4NJxD5tlqI
Start trimming:  E4NJxD5tlqI
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  E4NJxD5tlqI
Start downloading:  E4cX3kv2PG0
Finish downloading:  E4cX3kv2PG0
Start trimming:  E4cX3kv2PG0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimmi

Finish downloading:  GHO6-XMIFrY
Start trimming:  GHO6-XMIFrY
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  GHO6-XMIFrY
Start downloading:  GHsC4KKLUlQ
Finish downloading:  GHsC4KKLUlQ
Start trimming:  GHsC4KKLUlQ
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  GHsC4KKLUlQ
Start downloading:  GIy7xRfcymU
Finish downloading:  GIy7xRfcymU
Start trimming:  GIy7xRfcymU
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  GIy7xRfcymU
Start downloading:  GJNYIqWk2rA
Finish downloading:  GJNYIqWk2rA
Start trimming:  GJNYIqWk2rA
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  GJNYIqWk2rA
Start downloading:  GRWjtpd46-o
Finish downloading:  GRWjtpd46-o
Start trimming:  GRWjtpd46-o
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  GRWjtpd46-o
Start downloading:  GTBaQPFFghM
Finish downloading:  GTBaQPFFghM
St

Finish downloading:  J0QCHWK-ohk
Start trimming:  J0QCHWK-ohk
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  J0QCHWK-ohk
Start downloading:  J1OcQCjye9Q
Finish downloading:  J1OcQCjye9Q
Start trimming:  J1OcQCjye9Q
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  J1OcQCjye9Q
Start downloading:  J4jaChmZcZE
Finish downloading:  J4jaChmZcZE
Start trimming:  J4jaChmZcZE
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  J4jaChmZcZE
Start downloading:  J4p8Z_FXZzA
Finish downloading:  J4p8Z_FXZzA
Start trimming:  J4p8Z_FXZzA
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  J4p8Z_FXZzA
Start downloading:  J6imc-Qx_Gw
Finish downloading:  J6imc-Qx_Gw
Start trimming:  J6imc-Qx_Gw
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  J6imc-Qx_Gw
Start downloading:  J6n8FT9xcw8
Finish downloading:  J6n8FT9xcw8
St

Finish downloading:  LdPedHdfyzQ
Start trimming:  LdPedHdfyzQ
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  LdPedHdfyzQ
Start downloading:  LfYM3MjaCl0
Finish downloading:  LfYM3MjaCl0
Start trimming:  LfYM3MjaCl0
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  LfYM3MjaCl0
Start downloading:  Li3QnE0Ze-o
Error in download video: Li3QnE0Ze-o is a private video
Start downloading:  LkFa3qcljlg
Error in download video: 'streamingData'
Start downloading:  LnCXWHXL7ak
Finish downloading:  LnCXWHXL7ak
Start trimming:  LnCXWHXL7ak
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  LnCXWHXL7ak
Start downloading:  Loyjwq9bOTo
Finish downloading:  Loyjwq9bOTo
Start trimming:  Loyjwq9bOTo
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  Loyjwq9bOTo
Start downloading:  LwijRjr8ZN8
Finish downloading:  LwijRjr8ZN8
Start trimming:  LwijRjr8ZN8
Movie

Finish downloading:  QVzNj3k6ZUo
Start trimming:  QVzNj3k6ZUo
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  QVzNj3k6ZUo
Start downloading:  QaaqPb25eb4
Finish downloading:  QaaqPb25eb4
Start trimming:  QaaqPb25eb4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  QaaqPb25eb4
Start downloading:  QahWo5ESygM
Finish downloading:  QahWo5ESygM
Start trimming:  QahWo5ESygM
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  QahWo5ESygM
Start downloading:  QlYqu2EmouU
Finish downloading:  QlYqu2EmouU
Start trimming:  QlYqu2EmouU
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  QlYqu2EmouU
Start downloading:  QpRX-MvKsWY
Finish downloading:  QpRX-MvKsWY
Start trimming:  QpRX-MvKsWY
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  QpRX-MvKsWY
Start downloading:  QrY_Fd9PHtc
Finish downloading:  QrY_Fd9PHtc
St

Finish downloading:  Xx2lQ4M2D00
Start trimming:  Xx2lQ4M2D00
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  Xx2lQ4M2D00
Start downloading:  YLVOS4GvDeY
Finish downloading:  YLVOS4GvDeY
Start trimming:  YLVOS4GvDeY
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  YLVOS4GvDeY
Start downloading:  YUhyGd0JUdg
Finish downloading:  YUhyGd0JUdg
Start trimming:  YUhyGd0JUdg
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  YUhyGd0JUdg
Start downloading:  YYBzxoJr2oQ
Finish downloading:  YYBzxoJr2oQ
Start trimming:  YYBzxoJr2oQ
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  YYBzxoJr2oQ
Start downloading:  YiqWb9yBLoI
Finish downloading:  YiqWb9yBLoI
Start trimming:  YiqWb9yBLoI
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Finish trimming:  YiqWb9yBLoI
Start downloading:  YiwT3I6zWKo
Finish downloading:  YiwT3I6zWKo
St

# Extract features
## Create model

In [56]:
inception_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

## Extract features for `train` and `validation`

In [71]:
def get_id_by_path(full_path):
    return full_path.split('/')[-1].split('.')[0]

In [72]:
def generate_and_save_features(path, dataset, model):
    for target, item in zip(dataset.target, dataset.filenames):
        create_path(f'{path}/{dataset.target_names[target]}')
        
        frames = videoio.videoread(item, output_resolution=(299, 299))
        frames = np.array(frames)

        video_features = np.empty((0, 8, 8, 2048))
        for chunk in np.array_split(frames, 3):
            chunk_features = model(chunk)
            video_features = np.concatenate((video_features, chunk_features))
        
        name = get_id_by_path(item)
        np.save(f'{path}/{dataset.target_names[target]}/{name}.npy', video_features)

## Create dataset of sklearn

In [73]:
dataset_train = datasets.load_files(VIDEOS_PATH_TRAIN)
dataset_validation = datasets.load_files(VIDEOS_PATH_VALIDATE)

##  Execute model and save features
It generate and save features from this videos...

In [76]:
generate_and_save_features(FEATURES_PATH_TRAIN, dataset_train, inception_model)

In [77]:
generate_and_save_features(FEATURES_PATH_VALIDATION, dataset_validation, inception_model)